# Big Data Analytics Homework 02

*Complete this assignment in Google Colab. Prior to submitting a copy of this notebook (.ipynb format), run every cell and ensure you have corrected all runtime errors. Be sure to fill in your Name and SUID in the following cell. As always, you must do your own work. This means you may not use answers to the following questions generated by any other person or a generative AI tool such as ChatGPT. You may, however, discuss this assignment with others in a general way and seek help when you need it, but, again, you must do your own work.*

Name: Gughapriyaa Elango

SUID: 585992381

## Setup

In [1]:
! pip install pyspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import functions as fn

sc = SparkContext.getOrCreate()

spark = SparkSession\
    .builder\
    .appName('Homework 02')\
    .getOrCreate()

## RDDs

### Q1

Create a single-dimensional PySpark RDD named `bernoulli_rdd` that contains 1,000 Bernoulli probability distribution data points consisting of integers 0 or 1 with P(0) = P(1) = 0.5.

Use only PySpark RDDs and `map` to complete this question.

Hint: Use the [`RandomRDDs.uniformRDD`](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.mllib.random.RandomRDDs.html#pyspark.mllib.random.RandomRDDs.uniformRDD) function for your sampling step. But you will still need to map over this RDD with a custom function that creates 1's and 0's from the initial output.

In [4]:
# your code here
from pyspark.mllib.random import RandomRDDs

#creating bernoulli_rdd containing 1000 bernoulli probability distribn points:
bernoulli_rdd = RandomRDDs.uniformRDD(sc, 1000,seed=32)

#Function to make 1's and 0's:
def bernoulli_int(x):
    if x < 0.5:
        return 0
    else:
        return 1

#Map over RDD using created function:
bernoulli_rdd = bernoulli_rdd.map(bernoulli_int)

In [5]:
# do not modify
bernoulli_rdd.take(15)

[1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0]

### Q2

Using only Spark, count and display the number of 1's and the number of 0's in `bernoulli_rdd`.

Your output must be of the form "There are X 1's and Y 0's in bernoulli_rdd".

In [6]:
# your code here
count = bernoulli_rdd.countByValue() #Countbyvalue gives the count of each distinct value
print("There are ",count[1],"1's and ",count[0],"0's in bernoulli_rdd")

There are  491 1's and  509 0's in bernoulli_rdd


### Q3

Create a two new 2-dimensional RDD named `bernoulli_sample_rdd_1` and `bernoulli_sample_rdd_2` that each contain sample data from `bernoulli_rdd`. Each element of these RDDs should contain 10 samples (with replacement) from the original 1,000. The length of each RDD should be the number of samples which should be 50. In addition to the samples themselves, each data element in each RDD should contain `sample_number`, which should be calculated from each sample, not "hard-coded" as 10.

A sample element of the result will be of the form `[(sample_number, [sample])]`.

`bernoulli_sample_rdd_1` should be created using the [`sample`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.sample.html) method and `bernoulli_sample_rdd_2` should be created using the[`takeSample`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.takeSample.html) method.

In [19]:
no = 10 #number of sample
frac = no/1000 #fraction for fraction in sample method

bernoulli_sample_rdd_1 = [] #initalize empty list
for i in range(50):
  sample = bernoulli_rdd.sample(withReplacement = True, fraction = frac,seed=i).collect() #get sample
  leng = len(bernoulli_rdd.sample(withReplacement = True, fraction = frac,seed=i).collect()) #get length
  bernoulli_sample_rdd_1.append([leng,sample]) #create 50 [length,sample]

bernoulli_sample_rdd_1 = sc.parallelize(bernoulli_sample_rdd_1) #convert into rdd

In [40]:
bernoulli_sample_rdd_2 = [] #initalize empty list
for j in range(50):
  sample2 = bernoulli_rdd.takeSample(withReplacement = True, num=no, seed=j) #get sample
  leng2 = len(bernoulli_rdd.takeSample(withReplacement = True, num=no, seed=j)) #get length
  bernoulli_sample_rdd_2.append([leng2,sample2]) #create 50 [length,sample]

bernoulli_sample_rdd_2 = sc.parallelize(bernoulli_sample_rdd_2) #convert into rdd

In [20]:
# do not modify
bernoulli_sample_rdd_1.take(10)

[[5, [0, 0, 1, 0, 0]],
 [5, [1, 0, 0, 1, 0]],
 [10, [1, 1, 1, 0, 0, 0, 0, 0, 0, 1]],
 [10, [1, 0, 1, 1, 0, 0, 0, 0, 1, 0]],
 [11, [1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0]],
 [11, [1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1]],
 [12, [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1]],
 [12, [1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1]],
 [10, [1, 1, 0, 1, 0, 0, 0, 0, 0, 1]],
 [10, [0, 0, 0, 0, 0, 1, 0, 1, 0, 0]]]

In [41]:
# do not modify
bernoulli_sample_rdd_2.take(10)

[[10, [0, 1, 1, 0, 0, 1, 1, 1, 0, 1]],
 [10, [1, 0, 0, 1, 1, 1, 0, 0, 1, 1]],
 [10, [1, 1, 1, 1, 1, 0, 1, 0, 0, 0]],
 [10, [1, 1, 1, 0, 1, 1, 1, 0, 0, 0]],
 [10, [0, 1, 1, 0, 0, 1, 1, 1, 0, 0]],
 [10, [1, 0, 0, 0, 0, 0, 0, 1, 1, 0]],
 [10, [1, 1, 1, 0, 0, 1, 0, 1, 0, 1]],
 [10, [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]],
 [10, [0, 1, 1, 0, 1, 1, 1, 0, 1, 0]],
 [10, [1, 0, 1, 1, 0, 1, 1, 1, 0, 0]]]

### Q4

Explain key difference between `bernoulli_sample_rdd_1` and `bernoulli_sample_rdd_2` and the reason for it.

.sample doesnt returns the exact sample size as intended because internally Spark uses bernounlli sampling for taking the sample. Hence the number of samples doesnt remain fixed, the length varies in each sample. When using .takeSample we can get the exact number of samples as intended which is 10 each time, however this can be take longer time when RDD datasets are large.sample function when set to replacement = True, in the background of Pyspark uses poisson sampling which is a non-deterministic sample size.<br>
Reference : https://stackoverflow.com/questions/32837530/how-to-get-a-sample-with-an-exact-sample-size-in-spark-rdd
<br>
bernoulli_sample_rdd_1 returns an RDD when using .sample and bernoulli_sample_rdd_2 returns a list when using .takesample. Hence bernoulli_sample_rdd_2 needs to be converted into RDD from list in order to use map.

### Q5

Re-using code from above that created `bernoulli_sample_rdd_2`, create `bernoulli_sample_rdd_3` which has 100 observations per sample.

Using PySpark `map`, create a new RDD named `bernoulli_sample_mean_rdd` that contains the sampling distribution of the means of the samples contained in `bernoulli_sample_rdd_3`.

In [38]:
# your code here
bernoulli_sample_rdd_3 = [] #initialize list
for j in range(50):
  sample2 = bernoulli_rdd.takeSample(withReplacement = True, num=no, seed=j) #get sample
  bernoulli_sample_rdd_3.append([j,sample2]) #element number, sample

bernoulli_sample_mean_rdd = sc.parallelize(bernoulli_sample_rdd_3).map(lambda x: (x[0],sum(x[1])/len(x[1])))


In [39]:
# do not modify
bernoulli_sample_mean_rdd.take(10)

[(0, 0.6),
 (1, 0.6),
 (2, 0.6),
 (3, 0.6),
 (4, 0.5),
 (5, 0.3),
 (6, 0.6),
 (7, 0.1),
 (8, 0.6),
 (9, 0.6)]

## DataFrames

In this section we will work with data from the U.S. Environmental Protection Agency (EPA). There are two data sets. The first data set consists of daily **temperatures** collected at the U.S. **city** level. The second data set consists of daily **air quality** data collected at the U.S. **county** level. These measurements were taken for the full year 2021.

In [42]:
# download the temperature and aqi data sets
%%bash

if [[ ! -f us-daily-temperatures-2021.csv.csv ]]; then
 wget https://syr-bda.s3.us-east-2.amazonaws.com/us-daily-temperatures-2021.csv -q
fi

if [[ ! -f us-daily-aqi-2021.csv.csv ]]; then
 wget https://syr-bda.s3.us-east-2.amazonaws.com/us-daily-aqi-2021.csv -q
fi

### Q6

Load the temperature data (using Spark) into a data frame called `temperature`. Load the air quality data into a data frame called `aqi`. Print the schema and the number of rows for each data set.

In [43]:
# your code here
temperature = spark.read.format('csv')\
                .option('header', 'true')\
                .option('path', 'us-daily-temperatures-2021.csv')\
                .load()
#printSchema
temperature.printSchema()
print("Number of rows : ",temperature.count())

root
 |-- date: string (nullable = true)
 |-- state: string (nullable = true)
 |-- county: string (nullable = true)
 |-- city: string (nullable = true)
 |-- sites_reporting: string (nullable = true)
 |-- mean_temperature_f: string (nullable = true)
 |-- max_temp_f: string (nullable = true)
 |-- max_temp_hour: string (nullable = true)

Number of rows :  206292


In [44]:
aqi = spark.read.format('csv')\
                .option('header', 'true')\
                .option('path', 'us-daily-aqi-2021.csv')\
                .load()
#printSchema
aqi.printSchema()
print("Number of rows : ",aqi.count())

root
 |-- date: string (nullable = true)
 |-- state: string (nullable = true)
 |-- county: string (nullable = true)
 |-- aqi: string (nullable = true)
 |-- category: string (nullable = true)

Number of rows :  130922


The `temperature` data is reported at the city level, but the `aqi` data is at the county level. We want the "grain" of these data sets to match. This means we need to aggregate the the `temperature` data to the county level. This is tricky because we could possibly have counties in different states with the same name. This means you'll want to aggregate not just at the county level (by date), but at the state *and* county level (by date). We also have 4 different metrics to deal with: the total number of sites, a mean, a max, and one value — `max_temp_hour` — that corresponds to the value of another metric, `max_temp_f`.

### Q7

Create a new data frame called `temperature_county` that contains the **mean** temperature, the **max** temperature, and the **total** sites reporting, for each unique `date`, `state`, and `county` in the `temperature` data frame. The columns `mean_temperature_f`, `max_temp_f`, and `sites_reporting` should retain their names. Additionally, round the **mean** and **max** columns to the nearest whole number and cast them as integers.

In [45]:
# your code
from pyspark.sql.functions import sum,avg,max
from pyspark.sql.types import IntegerType
#groupby date, state and county, aggregate using average, sum
temperature_county = temperature.groupby('date','state','county').agg(avg('mean_temperature_f').cast(IntegerType()).alias('mean_temperature_f'),max('max_temp_f').cast(IntegerType()).alias('max_temp_f'),sum('sites_reporting').cast(IntegerType()).alias('sites_reporting'))

In [46]:
# do not modify
print('Rows in temperature_county:', temperature_county.count())
temperature_county.orderBy('date', 'state', 'county').show(10)

Rows in temperature_county: 138555
+----------+--------+--------------------+------------------+----------+---------------+
|      date|   state|              county|mean_temperature_f|max_temp_f|sites_reporting|
+----------+--------+--------------------+------------------+----------+---------------+
|2021-01-01| Alabama|            Escambia|                63|        68|              1|
|2021-01-01| Alabama|           Jefferson|                65|        73|              2|
|2021-01-01|  Alaska|              Denali|                 0|         6|              1|
|2021-01-01|  Alaska|Fairbanks North Star|                -9|        -4|              7|
|2021-01-01| Arizona|             Cochise|                39|        49|              1|
|2021-01-01| Arizona|            Coconino|                29|        35|              1|
|2021-01-01| Arizona|            Maricopa|                49|        65|              1|
|2021-01-01| Arizona|              Navajo|                29|        42|   

### Q8

Create a new data frame called `county_max_temp_hour` that reports the `max_temp_hour` at the same level of aggregation as `temperature_county` in the previous step. This means it should have the **same number of rows** as `temperature_county` and contain the same grouping fields, but only one metric, `max_temp_hour`. Once you have created this data frame, **left join** it to `temperature county` as a new data frame called `temperature_county_final`.

I've provided some starter code for you below. Fill in where you see `???` in order to complete the answer.

In [47]:
# your code
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc

# define a window spec - #create unique combination of state,county,date and order by max temp
window_spec = Window.partitionBy('state','county','date')\
  .orderBy(desc('max_temp_f')).

# add row number for each group
df = temperature\
  .withColumn('row_num', row_number()\
              .over(window_spec))

# filter to the first row using row_num
df2 = df.filter(fn.col('row_num') == 1)

# keep only the grouping columns and max_temp_hour
county_max_temp_hour = df2.select('state','county','date','max_temp_hour')

# join county_max_temp_hour to temperature_county
temperature_county_final = temperature_county.join(county_max_temp_hour,['state','county','date'],'left')

In [48]:
# do not modify
print('Rows in county_max_temp_hour:', county_max_temp_hour.count())
county_max_temp_hour.orderBy('date', 'state', 'county').show(10)

Rows in county_max_temp_hour: 138555
+--------+--------------------+----------+-------------+
|   state|              county|      date|max_temp_hour|
+--------+--------------------+----------+-------------+
| Alabama|            Escambia|2021-01-01|            0|
| Alabama|           Jefferson|2021-01-01|           13|
|  Alaska|              Denali|2021-01-01|            4|
|  Alaska|Fairbanks North Star|2021-01-01|           14|
| Arizona|             Cochise|2021-01-01|           12|
| Arizona|            Coconino|2021-01-01|           12|
| Arizona|            Maricopa|2021-01-01|           13|
| Arizona|              Navajo|2021-01-01|           14|
| Arizona|                Pima|2021-01-01|           15|
|Arkansas|             Pulaski|2021-01-01|           10|
+--------+--------------------+----------+-------------+
only showing top 10 rows



In [49]:
# do not modify
print('Rows in temperature_county_final:', temperature_county_final.count())
temperature_county_final.orderBy('date', 'state', 'county').show(10)

Rows in temperature_county_final: 138555
+--------+--------------------+----------+------------------+----------+---------------+-------------+
|   state|              county|      date|mean_temperature_f|max_temp_f|sites_reporting|max_temp_hour|
+--------+--------------------+----------+------------------+----------+---------------+-------------+
| Alabama|            Escambia|2021-01-01|                63|        68|              1|            0|
| Alabama|           Jefferson|2021-01-01|                65|        73|              2|           13|
|  Alaska|              Denali|2021-01-01|                 0|         6|              1|            4|
|  Alaska|Fairbanks North Star|2021-01-01|                -9|        -4|              7|           14|
| Arizona|             Cochise|2021-01-01|                39|        49|              1|           12|
| Arizona|            Coconino|2021-01-01|                29|        35|              1|           12|
| Arizona|            Maricopa|2

### Q9

Join `aqi` to `temperature_county_final` and call the resulting data frame `daily_county_measurements`. Then write code that produces the `date`, name of the `county`, `state`, and `aqi` value where the **highest recorded `aqi`** occurred in 2021. In the event of a tie, take the first instance. Your answer should take the following form:

"The highest recorded AQI value in 2021 occured on [date], in [county] County, [state], and had a value of [aqi]."

I have included some comments and starter code to help you out.

In [50]:
# your code here

# join aqi to temperature_county_final
# and call the resulting data frame daily_county_measurements
daily_county_measurements = temperature_county_final.join(aqi,['state','county','date'],'left')
# create a data frame with a single row
# that contains the highest recorded AQI value in 2021
daily_county_measurements = daily_county_measurements.withColumn("aqi",daily_county_measurements.aqi.cast(IntegerType()))
#max_aqi = daily_county_measurements.select(max(fn.col('aqi'))).collect()
highest_aqi = daily_county_measurements.agg(fn.max_by('state','aqi').alias('state'),fn.max_by('county','aqi').alias('county'),fn.max('aqi').alias('aqi'),fn.max_by('date','aqi').alias('date')).collect()

# extract values from the data frame needed for printing
#date = ''.join(row['date'] for row in highest_aqi)

# print the output as specified
print("The highest recorded AQI value in 2021 occured on ",highest_aqi[0].date, ", in ",highest_aqi[0].county," County, ",highest_aqi[0].state,", and had a value of ",highest_aqi[0].aqi,".")


The highest recorded AQI value in 2021 occured on  2021-09-14 , in  Tulare  County,  California , and had a value of  537 .


### Q10

Using a process similar to Q8, create a new data frame called `highest_temperates_by_state_2021` that contains one row per state, and shows the `date`, `state`, and `max_temp_f` for the **highest recorded temperature** in that state in 2021. In the case of ties, pick the earliest day of the year.

In [57]:
# your code here - orderby desc max_temp to get maximum temperature
# and ascending date to get earliest date in case of ties
window_spec = Window.partitionBy('state')\
  .orderBy(desc('max_temp_f'),('date'))
#.filter('date like "2021%"')
df = temperature_county_final.withColumn('row_num', row_number()\
              .over(window_spec))

# filter to the first row using row_num
highest_temperates_by_state_2021 = df.filter(fn.col('row_num') == 1)

In [56]:
# do not modify
highest_temperates_by_state_2021\
  .select('date', 'state', 'max_temp_f')\
  .orderBy(desc('max_temp_f'))\
  .show()

+----------+-----------------+----------+
|      date|            state|max_temp_f|
+----------+-----------------+----------+
|2021-07-10|       California|       124|
|2021-06-17|Country Of Mexico|       123|
|2021-06-17|          Arizona|       118|
|2021-07-10|           Nevada|       118|
|2021-06-28|           Oregon|       116|
|2021-06-29|       Washington|       116|
|2021-06-29|            Idaho|       115|
|2021-08-17|           Kansas|       115|
|2021-07-10|             Utah|       114|
|2021-07-29|      Mississippi|       110|
|2021-06-15|          Montana|       110|
|2021-06-12|       New Mexico|       110|
|2021-06-15|          Wyoming|       109|
|2021-06-11|            Texas|       107|
|2021-07-09|         Colorado|       105|
|2021-06-17|             Iowa|       104|
|2021-07-19|     North Dakota|       103|
|2021-06-20|         Oklahoma|       103|
|2021-06-17|         Nebraska|       102|
|2021-07-30|         Arkansas|       101|
+----------+-----------------+----